# ONNX 模型基本信息

In [ ]:
from functools import reduce
import sys
import onnx
from onnx.mapping import TENSOR_TYPE_TO_NP_TYPE
from collections import defaultdict

def inspect_onnx(model_path): 
    onnx_model = onnx.load(model_path)
    print(f"====== load model {model_path} done.")

    g = onnx_model.graph

    def process_node():
        op_counter = defaultdict(int)
        for node in g.node:
            name = node.name
            optype = name.split("/")[-1].split("_")[0]
            optype = node.op_type
            op_counter[optype] += 1

        print("*" * 50)
        for k, v in sorted(op_counter.items()):
            print(f"{k:40}:  {v}")
        print("*" * 50)
        print("Total OP num: ", reduce(lambda i, j: i + j, op_counter.values(), 0))

    def process_inout():
        print("/n", "*" * 50, "inputs:")
        for node in g.input:
            name = node.name
            dtype = str(TENSOR_TYPE_TO_NP_TYPE[node.type.tensor_type.elem_type])
            shape = [dim.dim_value for dim in node.type.tensor_type.shape.dim]
            print(f"{name:40}{dtype:<10}{shape}")
            
        print("/n", "*" * 50, "outputs:")
        for node in g.output:
            name = node.name
            dtype = str(TENSOR_TYPE_TO_NP_TYPE[node.type.tensor_type.elem_type])
            shape = [dim.dim_value for dim in node.type.tensor_type.shape.dim]
            print(f"{name:40}{dtype:<10}{shape}")

    def nodename_info():

        node_name_set = set(n.name for n in g.node) 
        valinfo_name_set = set(n.name for n in g.value_info)


        print(f"node  name len: {len(node_name_set)}") 
        print(f"vainf name len: {len(valinfo_name_set)}")

        a = node_name_set.update(valinfo_name_set)
        print(f"merge name len: {len(node_name_set)}")
    
    process_node()
    process_inout()
    nodename_info()

In [1]:
import numpy as np
import onnxruntime as ort
import datetime
def infer_onnx(model_path):
    # Load the ONNX model
    sess = ort.InferenceSession(model_path)
    
    feeds = {}
    for input in sess.get_inputs():
        # Get input name and shape
        input_name = input.name
        input_shape = input.shape
        numpy_type = TENSOR_TYPE_TO_NP_TYPE[input.type]
        # Generate random input
        input_data = np.random.random_sample(input_shape).astype(numpy_type)

        feeds[input_name] = input_data

    # Run the model
    start = datetime.datetime.now()
    result = sess.run(None, feeds)
    print(datetime.datetime.now() - start)

    print([a.shape for a in result])


# ONNX 模型构造

In [ ]:
import onnx 
from onnx import helper 
from onnx import TensorProto 
 
# input and output 
a = helper.make_tensor_value_info('a', TensorProto.FLOAT, [10, 10]) 
x = helper.make_tensor_value_info('x', TensorProto.FLOAT, [10, 10]) 
b = helper.make_tensor_value_info('b', TensorProto.FLOAT, [10, 10]) 
output = helper.make_tensor_value_info('output', TensorProto.FLOAT, [10, 10]) 
 
# Mul 
mul = helper.make_node('Mul', ['a', 'x'], ['c']) 
 
# Add 
add = helper.make_node('Add', ['c', 'b'], ['output']) 
 
# graph and model 
graph = helper.make_graph([mul, add], 'linear_func', [a, x, b], [output]) 
model = helper.make_model(graph) 
 
# save model 
onnx.checker.check_model(model) 
print(model) 
onnx.save(model, 'linear_func.onnx')



In [ ]:
import onnxruntime 
import numpy as np 
 
sess = onnxruntime.InferenceSession('linear_func.onnx') 
a = np.random.rand(10, 10).astype(np.float32) 
b = np.random.rand(10, 10).astype(np.float32) 
x = np.random.rand(10, 10).astype(np.float32) 
 
output = sess.run(['output'], {'a': a, 'b': b, 'x': x})[0] 
 
assert np.allclose(output, a * x + b)

# ONNX 模型修改

**NOTE**: 下面的例子将Add改成Sub，一般如果不是类似的OP，直接修改op_type可能会有问题

In [ ]:
import onnx 
model = onnx.load('linear_func.onnx') 
 
node = model.graph.node 
node[1].op_type = 'Sub' 
 
onnx.checker.check_model(model) 
print(model)
onnx.save(model, 'linear_func_2.onnx')

import onnxruntime 
import numpy as np 
 
sess = onnxruntime.InferenceSession('linear_func_2.onnx') 
a = np.random.rand(10, 10).astype(np.float32) 
b = np.random.rand(10, 10).astype(np.float32) 
x = np.random.rand(10, 10).astype(np.float32) 
 
output = sess.run(['output'], {'a': a, 'b': b, 'x': x})[0] 
 
assert np.allclose(output, a * x - b)

# ONNX 子图提取

In [ ]:
import onnx  
 
onnx.utils.extract_model('linear_func_2.onnx', 'mul.onnx', ['a','x'], ['c'])

In [ ]:
# Test
from pprint import pprint
from onnx.mapping import TENSOR_TYPE_TO_NP_TYPE,NP_TYPE_TO_TENSOR_TYPE

pprint(TENSOR_TYPE_TO_NP_TYPE)
pprint(NP_TYPE_TO_TENSOR_TYPE)

In [17]:
from rich.console import Console
from rich.table import Column, Table

console = Console()

table = Table(show_header=True, header_style="bold green")
table.add_column("Date", style="dim", width=12)
table.add_column("Title")
table.add_column("Production Budget", justify="right")
table.add_column("Box Office", justify="right")
table.add_row(
    "Dev 20, 2019", "Star Wars: The Rise of Skywalker", "$275,000,000", "$375,126,118"
)
table.add_row(
    "May 25, 2018",
    "[red]Solo[/red]: A Star Wars Story",
    "$275,000,000",
    "$393,151,347",
)
table.add_row(
    "Dec 15, 2017",
    "Star Wars Ep. VIII: The Last Jedi",
    "$262,000,000",
    "[bold]$1,332,539,889[/bold]",
)

console.print(table)

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Date         ┃ Title                             ┃ Production Budget ┃     Box Office ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ Dev 20, 2019 │ Star Wars: The Rise of Skywalker  │      $275,000,000 │   $375,126,118 │
│ May 25, 2018 │ Solo: A Star Wars Story           │      $275,000,000 │   $393,151,347 │
│ Dec 15, 2017 │ Star Wars Ep. VIII: The Last Jedi │      $262,000,000 │ $1,332,539,889 │
└──────────────┴───────────────────────────────────┴───────────────────┴────────────────┘

In [ ]:
import onnx 
from onnx import shape_inference, TensorProto


m = onnx.load("C:/Users/jianlong/win24_models/QDQ-Models/Model-G3-QDQ/Model-G3-QDQ-v1_1/Model_G3.quant.onnx")
inferred_model = shape_inference.infer_shapes(m)
onnx.save(inferred_model, "save.onnx")

In [2]:
a = "C:/Users/jianlong/win24_models/QDQ-Models/Model-G3-QDQ/Model-G3-QDQ-v1_1/Model_G3.quant.onnx"

infer_onnx(a)

InvalidGraph: [ONNXRuntimeError] : 10 : INVALID_GRAPH : Load model from C:/Users/jianlong/win24_models/QDQ-Models/Model-G3-QDQ/Model-G3-QDQ-v1_1/Model_G3.quant.onnx failed:This is an invalid model. Type Error: Type 'tensor(uint16)' of input parameter (cache_frames_zero_point) of operator (QuantizeLinear) in node (cache_frames_QuantizeLinear) is invalid.